# Algoritmo de redes neuronales convolucionales

# Librerias

In [1]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 4.9 MB/s eta 0:00:00


In [2]:
# librería para manejar las flexiones gramaticales en el idioma inglés.
!pip install inflect
# Manejo de gráficas
!pip install scikit-plot

In [3]:
 # librería Natural Language Toolkit, usada para trabajar con textos
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 26.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=b168af591d1dbb0a88798c333ec116e0baede9e2cf5f454e31222d667df72d5f
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


# Entendimiento de datos

Primero importaremos la base dedatos

In [7]:
datos = pd.read_csv("tipo2_entrenamiento_estudiantes.csv", sep=',', encoding = 'utf-8') #Importacion de datos

Se visualizan las columnas y se ve una pequeña muestra de los datos

In [8]:
print(datos.head())

                                              Review  Class
0  Muy buena atención y aclaración de dudas por p...      5
1  Buen hotel si están obligados a estar cerca de...      3
2  Es un lugar muy lindo para fotografías, visite...      5
3  Abusados con la factura de alimentos siempre s...      3
4  Tuvimos un par de personas en el grupo que rea...      3


In [ ]:
datos.info() #Ver tipo de datos de cada columna con numero de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7875 entries, 0 to 7874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  7875 non-null   object
 1   Class   7875 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.2+ KB


Podemos ver que no hay nulos ya que el numero de casillas no vacias es el mismo nyumero de filas.

In [9]:
from scipy import stats as st

textos = datos.copy()
textos['Conteo'] = [len(x) for x in textos['Review']]
#textos['Moda'] =
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Review']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Review']]

In [10]:
print(textos.head())

                                              Review  Class  Conteo  Max  Min
0  Muy buena atención y aclaración de dudas por p...      5     252   20    0
1  Buen hotel si están obligados a estar cerca de...      3     297   13    1
2  Es un lugar muy lindo para fotografías, visite...      5     104   12    1
3  Abusados con la factura de alimentos siempre s...      3     422   14    1
4  Tuvimos un par de personas en el grupo que rea...      3     419   14    1


## Preparacion de los datos

In [11]:
#Codigo para quitar caracteres que no hacen parte del codigo Ascci
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        else:
            print("Found None value in remove_non_ascii function")
    return new_words

#Codigo para transformar  los caracteres en minuscula
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    if words is None:
        return []

    new_words = []
    for word in words:
        if word is not None:
            new_word = word.lower()
            new_words.append(new_word)
    return new_words
#Codigo para eliminar puntuaciones innecesarias
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words
#Codigo para tokenizar los numeros para que tengan una representacion textual
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit() and word is not None:
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

#Codigo para eliminar palabras que no son nada relevantes en la evaluacion
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    # Lista de palabras de parada en español
    stop_words = set(["de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", "para", "con", "no", "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", "ya", "o", "este", "sí", "porque", "esta", "entre", "cuando", "muy", "sin", "sobre", "también", "me", "hasta", "hay", "donde", "quien", "desde", "todo", "nos", "durante", "todos", "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos", "e", "esto", "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", "tanto", "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", "ella", "estar", "estas", "algunas", "algo", "nosotros", "mi", "mis", "tú", "te", "ti", "tu", "tus", "ellas", "nosotras", "vosotros", "vosotras", "os", "mío", "mía", "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos", "suyas", "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", "vuestra", "vuestros", "vuestras", "esos", "esas", "estoy", "estás", "está", "estamos", "estáis", "están", "esté", "estés", "estemos", "estéis", "estén", "estaré", "estarás", "estará", "estaremos", "estaréis", "estarán", "estaría", "estarías", "estaríamos", "estaríais", "estarían", "estaba", "estabas", "estábamos", "estabais", "estaban", "estuve", "estuviste", "estuvo", "estuvimos", "estuvisteis", "estuvieron", "estuviera", "estuvieras", "estuviéramos", "estuvierais", "estuvieran", "estuviese", "estuvieses", "estuviésemos", "estuvieseis", "estuviesen", "estando", "estado", "estada", "estados", "estadas", "estad", "he", "has", "ha", "hemos", "habéis", "han", "haya", "hayas", "hayamos", "hayáis", "hayan", "habré", "habrás", "habrá", "habremos", "habréis", "habrán", "habría", "habrías", "habríamos", "habríais", "habrían", "había", "habías", "habíamos", "habíais", "habían", "hube", "hubiste", "hubo", "hubimos", "hubisteis", "hubieron", "hubiera", "hubieras", "hubiéramos", "hubierais", "hubieran", "hubiese", "hubieses", "hubiésemos", "hubieseis", "hubiesen", "habiendo", "habido", "habida", "habidos", "habidas", "soy", "eres", "es", "somos", "sois", "son", "sea", "seas", "seamos", "seáis", "sean", "seré", "serás", "será", "seremos", "seréis", "serán", "sería", "serías", "seríamos", "seríais", "serían", "era", "eras", "éramos", "erais", "eran", "fui", "fuiste", "fue", "fuimos", "fuisteis", "fueron", "fuera", "fueras", "fuéramos", "fuerais", "fueran", "fuese", "fueses", "fuésemos", "fueseis", "fuesen", "siendo", "sido", "sed", "tengo", "tienes", "tiene", "tenemos", "tenéis", "tienen", "tenga", "tengas", "tengamos", "tengáis", "tengan", "tendré", "tendrás", "tendrá", "tendremos", "tendréis", "tendrán", "tendría", "tendrías", "tendríamos", "tendríais", "tendrían", "tenía", "tenías", "teníamos", "teníais", "tenían", "tuve", "tuviste", "tuvo", "tuvimos", "tuvisteis", "tuvieron", "tuviera", "tuvieras", "tuviéramos", "tuvierais", "tuvieran", "tuviese", "tuvieses", "tuviésemos", "tuvieseis", "tuviesen", "teniendo", "tenido", "tenida", "tenidos", "tenidas", "tened"])
    new_words = [word for word in words if word.lower() not in stop_words]

    return new_words
#Codigo para ejecucion de los pasos anteriores
def preprocessing(words):
  words = remove_non_ascii(words)
  words = to_lowercase(words)
  words = replace_numbers(words)
  words = remove_punctuation(words)
  words = remove_stopwords(words)
  return words

In [12]:
datos['Review'] = datos['Review'].apply(contractions.fix) #Aplica la corrección de las contracciones

## Tokenizar

Al tokenizar, se divide cada una de las plabaras que quedaron del paso anterior y se creara una columna nueva con una lista por cada fila de las palabras separadas

In [13]:
datos['words'] = datos['Review'].apply(word_tokenize)
print(datos.head())

                                              Review  Class  \
0  Muy buena atención y aclaración de dudas por p...      5   
1  Buen hotel si están obligados a estar cerca de...      3   
2  Es un lugar muy lindo para fotografías, visite...      5   
3  Abusados con la factura de alimentos siempre s...      3   
4  Tuvimos un par de personas en el grupo que rea...      3   

                                               words  
0  [Muy, buena, atención, y, aclaración, de, duda...  
1  [Buen, hotel, si, están, obligados, a, estar, ...  
2  [Es, un, lugar, muy, lindo, para, fotografías,...  
3  [Abusados, con, la, factura, de, alimentos, si...  
4  [Tuvimos, un, par, de, personas, en, el, grupo...  


In [ ]:
datos['words'].dropna()

0       [Muy, buena, atención, y, aclaración, de, duda...
1       [Buen, hotel, si, están, obligados, a, estar, ...
2       [Es, un, lugar, muy, lindo, para, fotografías,...
3       [Abusados, con, la, factura, de, alimentos, si...
4       [Tuvimos, un, par, de, personas, en, el, grupo...
                              ...                        
7870    [Me, parece, buen, sistema, ,, agiliza, el, tr...
7871    [Fue, una, escapada, de, un, día, desde, el, c...
7872    [La, Plaza, de, la, Revolución, es, un, lugar,...
7873    [Es, la, segunda, ocasión, que, me, quedo, en,...
7874    [Llegamos, por, casualidad, a, Los, Mercaderes...
Name: words, Length: 7875, dtype: object

Con esto, podemos aplicar el mejor modelo mirando las palabbras mas escenciales

In [ ]:
datos['words'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 7875 entries, 0 to 7874
Series name: words
Non-Null Count  Dtype 
--------------  ----- 
7875 non-null   object
dtypes: object(1)
memory usage: 61.6+ KB


In [ ]:
datos['words1']=datos['words'].apply(preprocessing) #Aplica la eliminación del ruido
print(datos.head())

                                              Review  Class  \
0  Muy buena atención y aclaración de dudas por p...      5   
1  Buen hotel si están obligados a estar cerca de...      3   
2  Es un lugar muy lindo para fotografías, visite...      5   
3  Abusados con la factura de alimentos siempre s...      3   
4  Tuvimos un par de personas en el grupo que rea...      3   

                                               words  \
0  [Muy, buena, atención, y, aclaración, de, duda...   
1  [Buen, hotel, si, están, obligados, a, estar, ...   
2  [Es, un, lugar, muy, lindo, para, fotografías,...   
3  [Abusados, con, la, factura, de, alimentos, si...   
4  [Tuvimos, un, par, de, personas, en, el, grupo...   

                                              words1  
0  [buena, atencion, aclaracion, dudas, parte, se...  
1  [buen, hotel, si, estan, obligados, cerca, cen...  
2  [lugar, lindo, fotografias, visiten, selina, m...  
3  [abusados, factura, alimentos, siempre, echan,...  
4  [par, 

Este es el resultado de la base de datos aplicando todas las funciones, la culumna words1 es la columna que usaremos de ahora en adelante.

## Normalizar

Para normalizar, primero se deirva cada palabra, luego la siguiente funcion lematiza cada verbo. Al final se unen ambos resultados para tener los datos normalizados

In [36]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return lemmatized_words


def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas


datos['words'] = datos['words'].apply(stem_and_lemmatize)
print(datos.head())

                                              Review  Class  \
0  Muy buena atención y aclaración de dudas por p...      5   
1  Buen hotel si están obligados a estar cerca de...      3   
2  Es un lugar muy lindo para fotografías, visite...      5   
3  Abusados con la factura de alimentos siempre s...      3   
4  Tuvimos un par de personas en el grupo que rea...      3   

                                               words  \
0  [m,  , u,  , y,  ,  ,  , b,  , u,  , e,  , n, ...   
1  [b,  , u,  , e,  , n,  ,  ,  , h,  , o,  , t, ...   
2  [e,  , s,  ,  ,  , u,  , n,  ,  ,  , l,  , u, ...   
3  [a,  , b,  , u,  , s,  , a,  , d,  , o,  ,  , ...   
4  [t,  , u,  , v,  , i,  , m,  , o,  ,  ,  , u, ...   

                                               word1  
0  m u y   b u e n a   a t e n c i ó n   y   a c ...  
1  b u e n   h o t e l   s i   e s t á n   o b l ...  
2  e s   u n   l u g a r   m u y   l i n d o   p ...  
3  a b u s a d o   c o n   l a   f a c t u r a   ...  
4  t u v 

Se convierten los datos seleccionados en dataFrame

In [22]:
datos['words'] =datos['words'].apply(lambda x: ' '.join(map(str, x)))
print(datos)

                                                 Review  Class  \
0     Muy buena atención y aclaración de dudas por p...      5   
1     Buen hotel si están obligados a estar cerca de...      3   
2     Es un lugar muy lindo para fotografías, visite...      5   
3     Abusados con la factura de alimentos siempre s...      3   
4     Tuvimos un par de personas en el grupo que rea...      3   
...                                                 ...    ...   
7870  Me parece buen sistema, agiliza el transporte,...      4   
7871  Fue una escapada de un día desde el complejo, ...      4   
7872  La Plaza de la Revolución es un lugar emblemát...      3   
7873  Es la segunda ocasión que me quedo en los cuar...      1   
7874  Llegamos por casualidad a Los Mercaderes, un g...      5   

                                                  words  \
0     m u y   b u e n a   a t e n c i ó n   y   a c ...   
1     b u e n   h o t e l   s i   e s t á n   o b l ...   
2     e s   u n   l u g a r   

# Implementacion del modelo de redes neuronales convolucionales (CNN)

Las CNN son usadas principalmente en el procesamiento de datos estructurados en trama, como imágenes y, más recientemente, texto.

En el procesamiento de imágenes, las CNN utilizan filtros convolucionales para detectar características locales, como bordes y texturas, y luego combinan estas características para formar representaciones jerárquicas más abstractas de la imagen. Esto les permite aprender patrones complejos y realizar tareas como clasificación de imágenes, detección de objetos y segmentación semántica.

En el procesamiento de textos, las CNN se utilizan para capturar patrones locales en cadenas de palabras, esto nos permite la clasificación de sentimientos o la categorización de reseñas.



## Preparacion del modelo

En primer lugar seleccionamos las columnas de interes para el modelo, en este caso es words la cual fue modificada a lo largo del cuaderno y class

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Datos de entrada
X = datos['words']  # Características
y = datos['Class']   # Etiquetas



In [24]:
# Convertir las etiquetas a one-hot encoding
y = tf.keras.utils.to_categorical(y - 1, num_classes=5)  # Restar 1 para que las clases vayan de 0 a 4



In [25]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [26]:
# Tokenización y padding de las palabras
max_words = 10000
max_sequence_length = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_sequences, maxlen=max_sequence_length)



## Implementacion del modelo

In [27]:
# Definir el modelo CNN
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_words, 100, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [28]:
# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Entrenar el modelo
model.fit(
    X_train_pad,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_test_pad, y_test)
)


Epoch 1/5
99/99 [==============================] - 7s 74ms/step - loss: 1.2339 - accuracy: 0.4622 - val_loss: 1.3439 - val_accuracy: 0.3854
Epoch 2/5
99/99 [==============================] - 8s 86ms/step - loss: 1.1765 - accuracy: 0.4925 - val_loss: 1.3381 - val_accuracy: 0.3924
Epoch 3/5
99/99 [==============================] - 8s 76ms/step - loss: 1.1486 - accuracy: 0.5041 - val_loss: 1.3496 - val_accuracy: 0.4006
Epoch 4/5
99/99 [==============================] - 8s 79ms/step - loss: 1.0946 - accuracy: 0.5363 - val_loss: 1.3612 - val_accuracy: 0.3994
Epoch 5/5
99/99 [==============================] - 9s 88ms/step - loss: 1.0613 - accuracy: 0.5529 - val_loss: 1.3740 - val_accuracy: 0.3937


## Visualizacion de resultados

In [39]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Accuracy:", accuracy)

50/50 [==============================] - 1s 10ms/step - loss: 1.3740 - accuracy: 0.3937
Accuracy: 0.3936507999897003


# Conclusiones

**Pérdida en el conjunto de entrenamiento:** La pérdida en el conjunto de entrenamiento disminuye de aproximadamente 1.23 en la primera época a alrededor de 1.06 en la quinta época. Esto sugiere que el modelo está mejorando su capacidad para hacer predicciones precisas sobre los datos de entrenamiento a medida que avanza el entrenamiento.

**Precisión en el conjunto de entrenamiento:** La precisión en el conjunto de entrenamiento aumenta de aproximadamente 0.46 en la primera época a alrededor de 0.55 en la quinta época. Esto indica que el modelo está mejorando su capacidad para clasificar correctamente los datos de entrenamiento a lo largo del tiempo.

**Pérdida en el conjunto de validación:** La pérdida en el conjunto de validación aumenta ligeramente de alrededor de 1.34 en la primera época a alrededor de 1.37 en la quinta época. Esto sugiere que el modelo podría estar experimentando un cierto grado de sobreajuste a los datos de entrenamiento, ya que la pérdida en el conjunto de validación aumenta a medida que avanza el entrenamiento.

**Precisión en el conjunto de validación:** La precisión en entre 39-40% a lo largo de las épocas. Esto indica que el modelo no mejora significativamente su capacidad para clasificar correctamente los datos de validación a lo largo del tiempo, lo que podría ser un indicio de que el modelo no generaliza bien a datos transformados.